In [63]:
# ===============================================================
# 1. COLUNAS
# ===============================================================
CURSO_COL, SEMESTRE_COL = "Curso", "Semestre"
DISC_COL,  DOC_COL      = "Disciplina", "Docente"
DIA_COL,   INI_COL, FIM_COL, SALA_COL = "Dia", "Início", "Fim", "Sala"

# =========================
# 2. CARREGA / LIMPA
# =========================
import pandas as pd, json
from pathlib import Path
from datetime import date
hoje = date.today().strftime("%d/%m/%Y") 

df = pd.read_csv("2025_2s_grade.csv", encoding="utf-8-sig")

df[DOC_COL] = (df.groupby([DISC_COL, SEMESTRE_COL])[DOC_COL]
                 .transform(lambda s: s.ffill().bfill()))

dup_key = [DISC_COL, SEMESTRE_COL, DIA_COL, INI_COL, FIM_COL]
def squash(g):
    row = g.iloc[0].copy()
    row[SALA_COL]  = g[SALA_COL].dropna().iloc[0] if g[SALA_COL].notna().any() else pd.NA
    row[DOC_COL]   = g[DOC_COL].dropna().iloc[0]  if g[DOC_COL].notna().any() else ""
    row[CURSO_COL] = sorted(set(g[CURSO_COL]))
    return row
df_hor = (df.groupby(dup_key, as_index=False, sort=False)
            .apply(squash).reset_index(drop=True))

df_hor["CursosKey"] = df_hor[CURSO_COL].apply(tuple)
def turma(g):
    return pd.Series({
        "Cursos":   list(g[CURSO_COL].iloc[0]),
        "Horarios": g.apply(lambda r: {
            "dia": r[DIA_COL], "inicio": r[INI_COL],
            "fim": r[FIM_COL], "sala": r[SALA_COL] if pd.notna(r[SALA_COL]) else "—"
        }, axis=1).tolist(),
        "Docente":  g[DOC_COL].iloc[0] or "—"
    })
agg = (df_hor.groupby([DISC_COL, SEMESTRE_COL, "CursosKey"], as_index=False)
              .apply(turma))
records = [dict(Disciplina=r[DISC_COL], Semestre=r[SEMESTRE_COL],
                Cursos=r["Cursos"], Horarios=r["Horarios"], Docente=r["Docente"])
           for _, r in agg.iterrows()]
json_data = json.dumps(records, ensure_ascii=False)

# ===============================================================
# 3. HTML
# ===============================================================
html = """
<!DOCTYPE html><html lang='pt-BR'><head><meta charset='UTF-8'>
<title>Grade UFPR</title><meta name='viewport' content='width=device-width,initial-scale=1'>
<link href='https://fonts.googleapis.com/css2?family=Montserrat:wght@800&family=Roboto:wght@400;500&display=swap' rel='stylesheet'>
<style>
:root{--p:#1e40af;--bg:#f8fafc;--card:#fff;--tx:#1e293b;--lt:#64748b;--bd:#e2e8f0}
*{margin:0;padding:0;box-sizing:border-box;font-family:system-ui,-apple-system,sans-serif;color:var(--tx)}
body{background:var(--bg);padding:1rem}
.container{max-width:1200px;margin:0 auto}
h1{text-align:center;font:700 1.8rem/1.2 Montserrat,sans-serif;color:var(--p);margin-bottom:.8rem}
.subtitle{text-align:center;font-size:.9rem;color:var(--lt)}
.filters{display:flex;flex-wrap:wrap;gap:.6rem;justify-content:center;margin:1rem 0}
select,input[type=search]{padding:.44rem .8rem;border:1px solid var(--bd);border-radius:.45rem;font-size:.88rem;min-width:150px}
input[type=search]{flex:1;min-width:190px}
.grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(230px,1fr));gap:.8rem}
.card{background:var(--card);border:1px solid var(--bd);border-radius:.7rem;padding:.75rem;display:flex;flex-direction:column;gap:.4rem}
.nome{font:600 .9rem/1.3 Roboto,sans-serif;color:var(--p)}
.meta{font-size:.75rem;color:var(--lt)}
ul.meta{padding-left:1rem;margin-top:.15rem;list-style:disc}
/* ===== CALENDÁRIO ===== */
#calendarioBox{margin-top:1.3rem;display:none}
.calendario{display:grid;border:1px solid var(--bd)}
.calendario div{border:1px solid var(--bd);font-size:.7rem;overflow:hidden;display:flex;align-items:center}
.hora{background:var(--bg);justify-content:flex-end;padding-right:3px;font-weight:600}
.dia{background:var(--p);color:#fff;font-weight:600;justify-content:center}
.slot{background:transparent;height:22px;border-bottom:1px solid var(--bd)}
.evento{background:var(--p);color:#fff;padding:2px 4px;border-radius:.3rem;font-size:.68rem;display:flex;align-items:center;z-index:2;margin-bottom:1px}
.conflito{color:#b91c1c;font-size:.8rem;margin-top:.4rem;font-weight:600}
</style></head><body>
<div class='container'>
<h1>Grade de Disciplinas – UFPR (2s 2025)</h1>
<p class='subtitle'>Atualizado em ##DATE##
<div class='filters'>
<select id='curso'><option value=''>Todos os cursos</option></select>
<select id='semestre'><option value=''>Todos os semestres</option></select>
<select id='docente'><option value=''>Todos os docentes</option></select>
<select id='sala'><option value=''>Todas as salas</option><option value='_check'>Verificar erros</option></select>
<input type='search' id='busca' placeholder='Buscar disciplina…'>
</div>
<div id='list' class='grid'></div>
<div id='calendarioBox'><h2 style='font-size:1rem;margin-bottom:.3rem'>Calendário</h2>
<div id='calendario' class='calendario'></div><div id='conflicts'></div></div></div>
<script>
const data=JSON.parse(`##DATA##`);
const $=s=>document.querySelector(s);
const cursoSel=$('#curso'),semSel=$('#semestre'),docSel=$('#docente'),
      salaSel=$('#sala'),busca=$('#busca'),list=$('#list'),
      calBox=$('#calendarioBox'),cal=$('#calendario'),conf=$('#conflicts');
// preencher menus
[...new Set(data.flatMap(t=>t.Cursos))].sort().forEach(c=>cursoSel.insertAdjacentHTML('beforeend',`<option>${c}</option>`));
[...new Set(data.map(t=>t.Semestre))].sort((a,b)=>a-b).forEach(s=>semSel.insertAdjacentHTML('beforeend',`<option>${s}</option>`));
[...new Set(data.map(t=>t.Docente).filter(x=>x&&x!=='—'))].sort()
  .forEach(p=>docSel.insertAdjacentHTML('beforeend',`<option>${p}</option>`));
[...new Set(data.flatMap(t=>t.Horarios.map(h=>h.sala)).filter(x=>x&&x!=='—'))].sort()
  .forEach(s=>salaSel.insertAdjacentHTML('beforeend',`<option>${s}</option>`));
[cursoSel,semSel,docSel,salaSel].forEach(el=>el.onchange=render);busca.oninput=render;render();
function render(){
  const sala=salaSel.value,curso=cursoSel.value,sem=semSel.value,doc=docSel.value,txt=busca.value.toLowerCase();
  [cursoSel,semSel,docSel,busca].forEach(e=>e.disabled=!!sala&&sala!=='_check');
  let fil=data;
  if(sala&&!['_check',''].includes(sala)) fil=data.filter(t=>t.Horarios.some(h=>h.sala===sala));
  if(!sala){
    fil=data.filter(t=>(!curso||t.Cursos.includes(curso))&&(!sem||t.Semestre==sem)&&(!doc||t.Docente==doc)&&(!txt||t.Disciplina.toLowerCase().includes(txt)));
  }
  list.innerHTML= sala?'':fil.map(t=>{
    const hrs=t.Horarios.map(h=>`<li>${h.dia} ${h.inicio}–${h.fim} • ${h.sala}</li>`).join('');
    return `<div class='card'><div class='nome'>${t.Disciplina}</div>
            <div class='meta'>${t.Cursos.join(', ')} — ${t.Semestre}º</div>
            <div class='meta'><strong>${t.Docente}</strong></div><ul class='meta'>${hrs}</ul></div>`;
  }).join('');
  if(sala==='_check'){showConflicts();return;}
  if(sala|| (curso&&sem)){calBox.style.display='block';buildCal(fil);}else calBox.style.display='none';
}
function buildCal(arr){
  cal.innerHTML='';conf.innerHTML='';
  if(!arr.length) return;
  const toM=t=>+t.split(':')[0]*60+ +t.split(':')[1];
  let min=1e9,max=-1;arr.forEach(t=>t.Horarios.forEach(h=>{min=Math.min(min,toM(h.inicio));max=Math.max(max,toM(h.fim));}));
  min=Math.floor(min/30)*30;max=Math.ceil(max/30)*30;
  const rows=(max-min)/30,dias=['Seg','Ter','Qua','Qui','Sex'],col={Seg:2,Ter:3,Qua:4,Qui:5,Sex:6};
  cal.style.gridTemplateColumns='80px repeat(5,1fr)';
  cal.style.gridTemplateRows=`auto repeat(${rows},22px)`;
  cal.insertAdjacentHTML('beforeend',"<div></div>"+dias.map(d=>`<div class='dia'>${d}</div>`).join(''));
  for(let i=0;i<rows;i++){
    const m=min+i*30,h=(m/60|0).toString().padStart(2,'0')+':'+(m%60).toString().padStart(2,'0');
    cal.insertAdjacentHTML('beforeend',`<div class='hora' style='grid-row:${i+2}'>${h}</div>`+
      "<div class='slot'></div>".repeat(5));
  }
  const occ={}, add=(k,d)=>{(occ[k]=occ[k]||[]).push(d)};
  arr.forEach(t=>t.Horarios.forEach(h=>{
    const c=col[h.dia.slice(0,3)];if(!c) return;
    const r1=(toM(h.inicio)-min)/30+2,r2=(toM(h.fim)-min)/30+2;
    for(let m=toM(h.inicio);m<toM(h.fim);m+=30)add(`${h.sala}|${h.dia}|${m}`,t.Disciplina);
    cal.insertAdjacentHTML('beforeend',`<div class='evento' style='grid-column:${c};grid-row:${r1}/${r2}'>${t.Disciplina} • ${h.sala} • ${t.Docente}</div>`);
  }));
  const confl=Object.entries(occ).filter(([,v])=>v.length>1).map(([k,v])=>{
    const[s,d,m]=k.split('|'),h=(m/60|0).toString().padStart(2,'0')+':'+(m%60).toString().padStart(2,'0');
    return `${d} ${h} – Sala ${s}: ${[...new Set(v)].join(' × ')}`;});
  if(confl.length)conf.innerHTML="<div class='conflito'>⚠️ Conflitos:<br>"+confl.join('<br>')+"</div>";
}
function showConflicts(){
  calBox.style.display='block';cal.innerHTML='';conf.innerHTML='';
  const occ={},toM=t=>+t.split(':')[0]*60+ +t.split(':')[1];
  data.forEach(t=>t.Horarios.forEach(h=>{
    for(let m=toM(h.inicio);m<toM(h.fim);m+=30){
      const k=`${h.sala}|${h.dia}|${m}`;(occ[k]=occ[k]||[]).push(t.Disciplina);
    }}));
  const confl=Object.entries(occ).filter(([,v])=>v.length>1).map(([k,v])=>{
    const[s,d,m]=k.split('|'),h=(m/60|0).toString().padStart(2,'0')+':'+(m%60).toString().padStart(2,'0');
    return `${d} ${h} – Sala ${s}: ${[...new Set(v)].join(' × ')}`;});
  conf.innerHTML=confl.length?`<div class='conflito'>⚠️ Conflitos:<br>${confl.join('<br>')}</div>`:
                                  "<div class='meta'>Nenhum conflito encontrado.</div>";
}
</script></body></html>
""".replace("##DATA##", json_data)
html = html.replace("##DATE##", hoje)
Path("index.html").write_text(html, encoding="utf-8")


C:\Users\Dell\AppData\Local\Temp\ipykernel_22760\689219755.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .transform(lambda s: s.ffill().bfill()))
C:\Users\Dell\AppData\Local\Temp\ipykernel_22760\689219755.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_hor = (df.groupby(dup_key, as_index=False, sort=False)
C:\Users\Dell\AppData\Local\Temp\ipykernel_22760\689219755.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is de

43415